# Batch Depth Prediction

This Colab will allow you to import an unlimited amount of pictures (jpg or not jpg, many formats supported), and transform all of them into depth maps predictions, all at once. You can then download all of them as a zip file.

How to Use:

1.   Run the 1st cell to load dependencies and initialize folders in the Files pane 
2.   Right click "inputFolder" folder -> Import -> Select and load all your pictures 
3.   Run the 2nd cell to prepare the image processor
4.   Run the 3rd cell to generate the depth maps
5.   Download "outputFolder" as zip, by running the 4th and last cell

(credits: https://huggingface.co/spaces/nielsr/dpt-depth-estimation  )


# Run this cell first to initialize dependencies and folders


In [ ]:
!pip install transformers
!pip install torch
!pip install git+https://github.com/nielsrogge/transformers.git@add_dpt_redesign#egg=transformers
!pip install numpy
!pip install Pillow

!mkdir inputFolder
!mkdir outputFolder

# Run this cell next to prepare the image processor

In [ ]:
from transformers import DPTFeatureExtractor, DPTForDepthEstimation
import torch
import numpy as np
import os
from PIL import Image
from google.colab import files

torch.hub.download_url_to_file('http://images.cocodataset.org/val2017/000000039769.jpg', 'cats.jpg')

feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-large")
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large")

def process_image(image):
    # prepare image for the model
    encoding = feature_extractor(image, return_tensors="pt")
    
    # forward pass
    with torch.no_grad():
       outputs = model(**encoding)
       predicted_depth = outputs.predicted_depth
    
    # interpolate to original size
    prediction = torch.nn.functional.interpolate(
                        predicted_depth.unsqueeze(1),
                        size=image.size[::-1],
                        mode="bicubic",
                        align_corners=False,
                 ).squeeze()
    output = prediction.cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype('uint8')
    img = Image.fromarray(formatted)
    return img
    
    return result

# Run this next cell once you have imported all your pictures inside "inputFolder"

In [ ]:
for filename in os.listdir("inputFolder"):
  if not filename.startswith("."): 
    myimg = Image.open("inputFolder/" + filename, mode='r')
    processedimg = process_image(myimg)
    processedimg.save("outputFolder/" + filename)

# Download the results by running this cell


In [ ]:
!zip -r /content/outputFolder.zip /content/outputFolder
files.download('/content/outputFolder.zip')